In [1]:
!pip install transformers datasets peft accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 60.1 MB/s eta 0:00:0031m64.3 MB/s eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.0
    Uninstalling pyarrow-19.0.0:
      Successfully uninstalled pyarrow-19.0.0
  Attempting uninstall: dill0m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
    Found existing installation: dill 0.3.8━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
    Uninstalling dill-0.3.8:38;5;237m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
      Successfully uninstalled dill-0.3.8m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [evaluate]━━ 6/8 [peft]ets]ess]


import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import evaluate

In this step, we load the IMDB sentiment dataset from Hugging Face Datasets.
It has two labels:

- 0 = negative

- 1 = positive

In [3]:
dataset = load_dataset("imdb")
dataset

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

IMDB already includes train and test sets.
We will create a validation set by splitting 10% of the training set.

In [4]:
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
val_dataset = dataset["test"]
test_dataset = load_dataset("imdb", split="test")

We load the tokenizer for distilbert-base-uncased and apply it to the dataset.
Max length = 256 to reduce training time.

In [5]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

We load DistilBERT configured for classification (2 labels).

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LoRA injects small trainable matrices into the Transformer layers, allowing us to:

- Train fewer parameters

- Save GPU memory

- Train faster

We target the query and value projection matrices.

LoRA hyperparameters:

- r = 8

- alpha = 16

- dropout = 0.1

In [8]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

ValueError: Please specify `target_modules` or `target_parameters`in `peft_config`

In [9]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(dim=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels),
        "f1": f1.compute(predictions=preds, references=labels)
    }

We define the training configuration:

- Batch size: 16

- Epochs: 3

- FP16 enabled

- Saves best model

In [10]:
training_args = TrainingArguments(
    output_dir="./lora-distilbert-imdb",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

NameError: name 'training_args' is not defined

In [12]:
trainer.train()

NameError: name 'trainer' is not defined

We evaluate the fine-tuned model on the held-out test set.

In [13]:
results = trainer.evaluate(test_dataset)
results

NameError: name 'trainer' is not defined

In [14]:
trainer.save_model("./lora-distilbert-imdb-final")

NameError: name 'trainer' is not defined

In [15]:
def predict(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return "Positive" if pred == 1 else "Negative"

predict("This movie was amazing, I loved it!")

'Negative'